In [2]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append('..')
from aips import *
import pandas as pd
import os
from IPython.core.display import display,HTML
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("aips-ch4-getting-started-retrotech").getOrCreate()

In [3]:
#Get datasets
![ ! -d 'retrotech' ] && git clone https://github.com/ai-powered-search/retrotech.git
! cd retrotech && git pull
! cd retrotech && tar -xvf products.tgz -C '../../data/retrotech/' && tar -xvf signals.tgz -C '../../data/retrotech/'

Already up to date.
products.csv
signals.csv


In [4]:
! cd ../data/retrotech/ && head products.csv

"upc","name","manufacturer","shortDescription","longDescription"
"096009010836","Fists of Bruce Lee - Dolby - DVD",\N,\N,\N
"043396061965","The Professional - Widescreen Uncut - DVD",\N,\N,\N
"085391862024","Pokemon the Movie: 2000 - DVD",\N,\N,\N
"067003016025","Summerbreeze - CD","Nettwerk",\N,\N
"731454813822","Back for the First Time [PA] - CD","Def Jam South",\N,\N
"024543008200","Big Momma's House - Widescreen - DVD",\N,\N,\N
"031398751823","Kids - DVD",\N,\N,\N
"037628413929","20 Grandes Exitos - CD","Sony Discos Inc.",\N,\N
"060768972223","Power Of Trinity (Box) - CD","Sanctuary Records",\N,\N


In [5]:
#Create Products Collection
products_collection="products"
create_collection(products_collection)
enable_ltr(products_collection)

#Modify Schema to make some fields explicitly searchable by keyword
upsert_text_field(products_collection, "upc")
upsert_text_field(products_collection, "name")
upsert_text_field(products_collection, "longDescription")
upsert_text_field(products_collection, "shortDescription")
upsert_text_field(products_collection, "manufacturer")
upsert_text_field(products_collection, "collectionName")

clear_copy_fields(products_collection)

ngram_analyzer = {
                "tokenizer": {
                    "class": "solr.StandardTokenizerFactory"
                },
                "filters":
                    [{"class": "solr.LowerCaseFilterFactory"},
                     {"class": "solr.NGramFilterFactory",
                      "minGramSize": "3",
                      "maxGramSize": "6",
                      #"preserveOriginal": "true"
                      }
                    ]
            }

text_general_analyzer = {
                "tokenizer": {
                    "class": "solr.StandardTokenizerFactory"
                },
                "filters":
                    [{"class": "solr.LowerCaseFilterFactory"}
                    ]
            }


add_text_field_type(products_collection,
                    analyzer=ngram_analyzer,
                    name="ngram",
                    omitTermFreqAndPositions=True)

add_text_field_type(products_collection,
                    analyzer=text_general_analyzer,
                    name="omit_norms",
                    omitNorms=True)



add_copy_field(products_collection, "name", "name_ngram")
add_copy_field(products_collection, "name", "name_omit_norms")
add_copy_field(products_collection, "name", "name_txt_en_split")


promoted = [27242815414,
 600603141003,
 27242813908,
 803238004525,
 27242799127,
 36725236271,
 883393003458,
 600603135088,
 9781400532711,
 97360810042,
 97360810042,
 97360810042,
 97360810042,
 803238004525,
 27242799127,
 36725236271,
 883393003458,
 36725236271,
 883393003458,
 27242815414,
# promoted transformers movies for example
 97360724240,
 97360722345,
 97368920347,
]

promoted = [{'upc': promoted_upc, 'promotion_b': True} for promoted_upc in promoted]

# Any extra fields we want to add manually
enriched_data = spark.createDataFrame(promoted)

print("Loading Products...")
csvFile = "../data/retrotech/products.csv"
product_update_opts={"zkhost": "aips-zk", "collection": products_collection, "gen_uniq_key": "true", "commit_within": "5000"}
csvDF = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(csvFile)
joined = csvDF.join(enriched_data, ['upc'], "left")
joined.write.format("solr").options(**product_update_opts).mode("overwrite").save()
print("Products Schema: ")
joined.printSchema()
print("Status: Success")

Wiping 'products' collection
[('action', 'CREATE'), ('name', 'products'), ('numShards', 1), ('replicationFactor', 1)]
Creating products' collection
Status: Success
Del/Adding LTR QParser for products collection
<Response [400]>
Status: Failure; Response:[ {'responseHeader': {'status': 400, 'QTime': 17}, 'errorMessages': ['error processing commands\n'], 'WARNING': 'This response format is experimental.  It is likely to change in the future.', 'error': {'metadata': ['error-class', 'org.apache.solr.api.ApiBag$ExceptionWithErrObject', 'root-error-class', 'org.apache.solr.api.ApiBag$ExceptionWithErrObject'], 'details': [{'delete-queryparser': 'ltr', 'errorMessages': ["NO such queryParser 'ltr' "]}], 'msg': 'error processing commands', 'code': 400}} ]
Status: Success
Adding LTR Doc Transformer for products collection
Status: Failure; Response:[ {'responseHeader': {'status': 400, 'QTime': 4}, 'errorMessages': ['error processing commands\n'], 'WARNING': 'This response format is experimental.  

/usr/local/spark/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


Loading Products...
Products Schema: 
root
 |-- upc: long (nullable = true)
 |-- name: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- shortDescription: string (nullable = true)
 |-- longDescription: string (nullable = true)
 |-- promotion_b: boolean (nullable = true)

Status: Success


In [5]:
query = "ipod"

collection = "products"
request = {
    "query": query,
    "fields": ["upc", "name", "manufacturer", "score"],
    "limit": 5,
    "params": {
      "qf": "name manufacturer longDescription",
      "defType": "edismax",
      "sort": "score desc, upc asc"
    }
}

search_results = requests.post(solr_url + collection + "/select", json=request).json()["response"]["docs"]
display(HTML(render_search_results(query, search_results)))

In [7]:
! cd ../data/retrotech && head signals.csv

"query_id","user","type","target","signal_time"
"u2_0_1","u2","query","nook","2019-07-31 08:49:07.3116"
"u2_1_2","u2","query","rca","2020-05-04 08:28:21.1848"
"u3_0_1","u3","query","macbook","2019-12-22 00:07:07.0152"
"u4_0_1","u4","query","Tv antenna","2019-08-22 23:45:54.1030"
"u5_0_1","u5","query","AC power cord","2019-10-20 08:27:00.1600"
"u6_0_1","u6","query","Watch The Throne","2019-09-18 11:59:53.7470"
"u7_0_1","u7","query","Camcorder","2020-02-25 13:02:29.3089"
"u9_0_1","u9","query","wireless headphones","2020-04-26 04:26:09.7198"
"u10_0_1","u10","query","Xbox","2019-09-13 16:26:12.0132"


## Download query sessions

Download simulated raw clickstream data

In [7]:
from ltr import download
simulated_queries = ['dryer', 'bluray', 'blue ray', 'headphones', 'ipad', 'iphone',
                     'kindle', 'lcd tv', 'macbook', 'nook', 'star trek', 'star wars',
                     'transformers dark of the moon']

sessions = [f"https://github.com/ai-powered-search/retrotech/raw/master/sessions/{query}_sessions.gz"
            for query in simulated_queries]


           
download(sessions, dest='data/')

GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/dryer_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/bluray_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/blue ray_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/headphones_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/ipad_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/iphone_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/kindle_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/lcd tv_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/macbook_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/nook_sessions.gz
GET https://github.com/ai-powered-search/retrotech/raw/master/sessions/star trek_sessions.gz
GET h

In [8]:
!pwd

/tmp/notebooks/ch12


In [9]:
!ls data/

'blue ray_sessions.gz'	 'lcd tv_sessions.gz'
 bluray_sessions.gz	  macbook_sessions.gz
 dryer_sessions.gz	  nook_sessions.gz
 headphones_sessions.gz  'star trek_sessions.gz'
 ipad_sessions.gz	 'star wars_sessions.gz'
 iphone_sessions.gz	 'transformers dark of the moon_sessions.gz'
 kindle_sessions.gz


In [10]:
!ls

 0.ch12.setup.ipynb  '1.AB Testing.ipynb'   data   retrotech


In [11]:
!ls retrotech/sessions/

'blue ray_sessions.gz'	 'lcd tv_sessions.gz'
 bluray_sessions.gz	  macbook_sessions.gz
 dryer_sessions.gz	  nook_sessions.gz
 headphones_sessions.gz  'star trek_sessions.gz'
 ipad_sessions.gz	 'star wars_sessions.gz'
 iphone_sessions.gz	 'transformers dark of the moon_sessions.gz'
 kindle_sessions.gz
